In [1]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
import pandas as pd
import random
import math

from pandas.core.frame import DataFrame


class Node:
    def __init__(self):
        self.inEdgeList = []
        self.outEdgeList = []
        self.value = None
        self.delta = None
    
    def addInEdge(self, edge):
        self.inEdgeList.append(edge)
    
    def addOutEdge(self, edge):
        self.outEdgeList.append(edge)

        
class InputNode(Node):
    pass

class OutputNode(Node):
    pass

class HiddenNode(Node):
    pass

class Edge:
    def __init__(self, fromNode, toNode):
        self.fromNode = fromNode
        self.toNode = toNode
        self.weight = None


class Layer:
    def __init__(self):
        self.nodes = []
    
    def addNode(self, node):
        self.nodes.append(node)
    def removeNode(self, node):
        self.nodes.remove(node)

class InputLayer(Layer):
    pass

class OutputLayer(Layer):
    pass

class HiddenLayer(Layer):
    pass

class Utility:
    def logistic(node: Node) -> float:
        sumValue = 0.0
        for edge in node.inEdgeList:            
            sumValue += edge.weight * edge.fromNode.value
        ans = 1/ (1 + math.exp(-sumValue))
        return ans

        

class Graph:
    def __init__(self):
        self.inputLayer = InputLayer()
        self.outputLayer = OutputLayer()
        self.hiddenLayerList = []
        self.nodeList = []
        self.edgeList = []
        self.trainDf = None
        self.testDf = None
        self.maxAttribList = []
        self.minAttribList = []
        self.targetList = []
        self.learningRate = 0.01
    
    def createHiddenLayer(self):
        layer = HiddenLayer()        
        self.hiddenLayerList.append(layer)    
    
    def createInputNode(self):
        node = InputNode()
        self.inputLayer.addNode(node)
        self.nodeList.append(node)
        return node
    
    def createOutputNode(self):
        node = OutputNode()
        self.outputLayer.addNode(node)
        self.nodeList.append(node)
        return node
    
    def createHiddenNode(self, hiddenLayer: HiddenLayer):
        node = HiddenNode()
        hiddenLayer.addNode(node)
        self.nodeList.append(node)
        return node

    def createMultipleInputNodes(self, count : int):        
        for i in range(count):
            self.createInputNode()            

    def createMultipleOutputNodes(self, count : int):        
        for i in range(count):
            self.createOutputNode()            

    def createMultipleHiddenNodes(self, hiddenLayer : HiddenLayer, count : int):        
        for i in range(count):
            self.createHiddenNode(hiddenLayer)

    def connectInputToHidden(self):
        for fromNode in self.inputLayer.nodes:
            for toNode in self.hiddenLayerList[0].nodes:
                edge = Edge(fromNode, toNode)
                self.edgeList.append(edge)
                fromNode.addOutEdge(edge)
                toNode.addInEdge(edge)

    def connectHiddenToOutput(self):
        for fromNode in self.hiddenLayerList[0].nodes:
            for toNode in self.outputLayer.nodes:
                edge = Edge(fromNode, toNode)
                self.edgeList.append(edge)
                fromNode.addOutEdge(edge)
                toNode.addInEdge(edge)
            
    def calculateInitialWeights(self):
        for edge in self.edgeList:
            edge.weight = random.uniform(0, 1/10)
    
    def printEdgeData(self):
        for edge in self.edgeList:
            edge.printData()

    def readDf(self, dataframe : DataFrame):
        # self.trainDf = dataframe
        self.trainDf = dataframe.sample(frac=0.8)
        self.testDf = dataframe.drop(self.trainDf.index)    
        self.trainDf.reset_index(drop=True, inplace=True)
        self.testDf.reset_index(drop=True, inplace=True)     
        # Read Max, Min value of Each column and store in a List                
        for i in range(self.trainDf.shape[1]-1):
            self.maxAttribList.append(pd.DataFrame.max(self.trainDf.iloc[:,[i]]))
            self.minAttribList.append(pd.DataFrame.min(self.trainDf.iloc[:,[i]]))

    def inputLayerFeed(self, row : DataFrame):        
        x = 0
        for node in self.inputLayer.nodes:
            normalized = float((row[x]-self.minAttribList[x])/(self.maxAttribList[x] - self.minAttribList[x]))
            node.value = normalized
            x+=1
        self.targetList = [0] * len(self.outputLayer.nodes)
        self.targetList[row[-1]-1] = 1        
    
    def softMax(self):
        denominator = self.softMaxDenominator()
        for node in self.outputLayer.nodes:
            node.value = math.exp(node.value)/denominator
            
    def softMaxDenominator(self) -> float:
        denominator = 0.0
        for node in self.outputLayer.nodes:
            denominator += math.exp(node.value)
        return denominator

    def mse(self):
        sumVal = 0.0
        m = len(self.outputLayer.nodes)
        for i in range(m):        
            sumVal += (self.targetList[i] - self.outputLayer.nodes[i].value) ** 2
        ans = sumVal/m
#         print("mse:",ans)
        return ans

    def deltaForOutputLayer(self):
        m = len(self.outputLayer.nodes)
        for i in range(m):
            node = self.outputLayer.nodes[i]
            deltaVal = node.value * (1-node.value) * (self.targetList[i]-node.value)
            node.delta = deltaVal
            
    def deltaForHiddenLayer(self):
        m = len(self.hiddenLayerList[0].nodes)
        for i in range(m):
            node = self.hiddenLayerList[0].nodes[i]
            sumVal = 0.0
            for edge in node.outEdgeList:
                sumVal += edge.toNode.delta * edge.weight
            deltaVal = node.value * (1-node.value) * sumVal
            node.delta = deltaVal

    def updateHiddenToOutputWeights(self):
        m = len(self.hiddenLayerList[0].nodes)
        for i in range(m):
            node = self.hiddenLayerList[0].nodes[i]
            for edge in node.outEdgeList:
                edge.weight += self.learningRate * edge.toNode.delta * node.value
    
    def updateInputToHiddenWeights(self):
        m = len(self.inputLayer.nodes)
        for i in range(m):
            node = self.inputLayer.nodes[i]
            for edge in node.outEdgeList:
                edge.weight += self.learningRate * edge.toNode.delta * node.value
    
    def singlePass(self, rowNumber):
        self.inputLayerFeed(self.trainDf.iloc[rowNumber])
        for layer in self.hiddenLayerList:
            for node in layer.nodes:
                node.value = Utility.logistic(node)
        
        for node in self.outputLayer.nodes:
            node.value = Utility.logistic(node)
        
        self.softMax()
        # self.mse()
        self.deltaForOutputLayer()
        self.deltaForHiddenLayer()
        self.updateHiddenToOutputWeights()
#         self.updateInputToHiddenWeights()
        
    def trainDfTest(self):
        rightAnswerCount = 0
        for i in range(self.testDf.shape[0]):
            self.inputLayerFeed(self.testDf.iloc[i])
            for layer in self.hiddenLayerList:
                for node in layer.nodes:
                    node.value = Utility.logistic(node)

            for node in self.outputLayer.nodes:
                node.value = Utility.logistic(node)
                
            outputList = [node.value for node in self.outputLayer.nodes]
            outputIndex = outputList.index(max(outputList))
            targetIndex = self.targetList.index(max(self.targetList))

            print("------------------------------------------------------")
            print("outputList",outputList)
            print("self.targetList",self.targetList)
            print("Target Index : ", targetIndex)
            print("Output Index:", outputIndex)
            print("Output Difference : ", 1-outputList[outputIndex])
            if outputIndex == targetIndex:
                rightAnswerCount+=1
        print("rightAnswerCount : ",rightAnswerCount, "/Out of : ",self.testDf.shape[0])

    def runANN(self):
#         print("------------------------------------------------")
#         icount = 0
#         for node in self.inputLayer.nodes:    
#             for edge in node.outEdgeList:
#                 print(icount," input edge weight:",edge.weight)
#                 icount+=1
#         icount = 0
#         print("------------------------------------------------")
#         for node in self.hiddenLayerList[0].nodes:    
#             for edge in node.outEdgeList:
#                 print(icount," hidden edge weight:",edge.weight)
#                 icount+=1
        
        for k in range(20):                                                          
            for i in range(self.trainDf.shape[0]):
                self.singlePass(i)
        
        # Last Pass
#         for i in range(self.trainDf.shape[0]):
#             self.singlePass(i)
#             self.mse()

#         icount = 0
#         print("------------------------------------------------")
#         for node in self.inputLayer.nodes:    
#             for edge in node.outEdgeList:
#                 print(icount,"input edge weight:",edge.weight)
#                 icount+=1
#         icount = 0
#         print("------------------------------------------------")
#         for node in self.hiddenLayerList[0].nodes:    
#             for edge in node.outEdgeList:
#                 print(icount," hidden edge weight:",edge.weight)  
#                 icount+=1

In [2]:
# from MLPv1 import *
import numpy as np
import pandas as pd
import re

def read_file(filepath) -> list:
    dataset = []        
    with open(filepath) as fp:
        for line in fp:                              
            compiler = re.compile("\d+")
            dataList = compiler.findall(line)                        
            row = list(map(int, dataList[1:]))            
            # print(row)
            dataset.append(row)
    return dataset

def createDataFrame(dataset : list):
    df = pd.DataFrame(dataset, columns=['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','label'])    
    return df


dataset = read_file("dataset.txt")
df = createDataFrame(dataset)
numberOfInputNodes = 10
numberOfOutputNodes = 8
g = Graph()
g.createHiddenLayer()
g.createMultipleInputNodes(numberOfInputNodes)
g.createMultipleHiddenNodes(g.hiddenLayerList[0], 10)
g.createMultipleOutputNodes(numberOfOutputNodes)

print("Graph total nodes : ",len(g.nodeList))
print("Input Nodes : ", len(g.inputLayer.nodes))
print("Output Nodes : ", len(g.outputLayer.nodes))
print("Hidden Nodes : ", len(g.hiddenLayerList[0].nodes))

g.connectInputToHidden()
g.connectHiddenToOutput()
g.calculateInitialWeights()
g.readDf(df)

# g.singlePass()

# for node in g.inputLayer.nodes:    
#     print("input node value:",node.value)

# print("------------------------------------------------")
# for node in g.hiddenLayerList[0].nodes:    
#     print("hidden node value:",node.value)

# print("------------------------------------------------")
# for node in g.outputLayer.nodes:    
#     print("output node value:",node.value)

# g.updateHiddenToOutputWeights()
# g.updateInputToHiddenWeights()
# g.singlePass()
# print("------------------------------------------------")
# for node in g.inputLayer.nodes:    
#     print("input node value:",node.value)

# print("------------------------------------------------")
# for node in g.hiddenLayerList[0].nodes:    
#     print("hidden node value:",node.value)

# print("------------------------------------------------")
# for node in g.outputLayer.nodes:    
#     print("output node value:",node.value)


Graph total nodes :  28
Input Nodes :  10
Output Nodes :  8
Hidden Nodes :  10


In [3]:
g.runANN()

In [4]:
g.trainDfTest()

------------------------------------------------------
outputList [0.5892295406511892, 0.6414430224750961, 0.6718737421120875, 0.6125672578717892, 0.6481383399663943, 0.5008820375770421, 0.4693017204277201, 0.4644347838804859]
self.targetList [0, 0, 0, 0, 1, 0, 0, 0]
Target Index :  4
Output Index: 2
Output Difference :  0.3281262578879125
------------------------------------------------------
outputList [0.5934498737850454, 0.6477816465492326, 0.6792584656552461, 0.618006860409724, 0.6547044239207342, 0.5010226324346786, 0.46784146369380525, 0.46263721882956554]
self.targetList [0, 0, 1, 0, 0, 0, 0, 0]
Target Index :  2
Output Index: 2
Output Difference :  0.32074153434475394
------------------------------------------------------
outputList [0.5928742287746699, 0.6472203641273918, 0.6786077771637513, 0.6172666136447874, 0.6540784707572149, 0.5008748281858342, 0.46798813529686994, 0.46291586877726615]
self.targetList [0, 0, 1, 0, 0, 0, 0, 0]
Target Index :  2
Output Index: 2
Output Dif